In [2]:
import numpy as np
import torch
from torch.utils.data import DataLoader , Dataset 

import matplotlib.pyplot as plt
from cfg import *


use constantpad2d() to resize 

In [4]:

#  currently using only nonPR (T1c)
import os
#non_T1C_data = "./data/PSPF_voc_data/non_PR/T1c/JPEGImages/" 
#non_T1C_target = "./data/PSPF_voc_data/non_PR/T1c/SegmentationClassPNG/" 

name_set = set()
total_data_path = [non_T1C_data , PR_T1C_data , non_T1_data , PR_T1_data , non_Flair_data , PR_Flair_data , non_T2_data , PR_T2_data]
for folder_name in total_data_path :
    for name in os.listdir(folder_name) :
        name = name.strip('.jpg').strip('.json')
        if name == 'ipynb_checkpoint' :
            continue
        name_set.add(name)   
#print(name_set)


name_set2 = set()
total_target_path = [PR_T1C_target , non_T1C_target , PR_T1_target , non_T1_target , PR_Flair_target , non_Flair_target , PR_T2_target , non_T2_target]
for folder_name2 in total_target_path :
    for name in os.listdir(folder_name2):
        name = name.strip('.png')
        if name == 'ipynb_checkpoint':
            continue
        name_set2.add(name)
print(name_set2)


#same patients set (make sure no mismatch)
if name_set2 == name_set :
    print('success')




{'14041674', '29285707', '29729120', '15525495', '30871154', '28267864', '22296689', '27607264', '30920239', '16113441', '15176128', '31046225', '80190941', '28248060', '31445157', '29301889', '30268702', '11775010', '17244854', '13488672', '13661917', '28358688', '26434694', '12261283', '11009708', '18456622', '25825974', '26082459', '15552568', '26658418', '20947834', '27931321', '26608196', '19425238', '31001430', '27472711', '21372392', '26073373', '29177415', '29816081', '21381973', '24237245', '30517836', '27710844', '31406726', '30732894', '25385854', '27103788', '27237207', '21105320', '10027124', '18765851', '20483981', '19315335', '33056266', '52847319'}
success


In [5]:
import cv2
import torchvision.transforms as T 

class Total_dataset(Dataset):
    def __init__(self, inputs: list, targets: list, transform=None) -> None:
        super().__init__()
        self.inputs = inputs
        self.targets = targets
        self.transform = transform
    
    def __len__(self):
        return len(self.inputs)
    def __getitem__(self , index : int  , transform = None):
        #是否有辦法用path + file name取用data
        input_img = self.inputs[index]
        target_img = self.targets[index]
        
        x_data , y_data = cv2.imread(input_img) , cv2.imread(target_img)

        if self.transform != None :
            #x_data , y_data = self.transform(x_data , y_data)
            for t in self.transform :
                if t == T.Normalize() :
                    pass
                else :
                    x_data = t(x_data)
                    y_data =t(y_data)
                
        
        if x_data.shape != [512 , 512 , 3] :
            x_data = cv2.resize(x_data , (512 , 512))
        if y_data.shape != [512 , 512 , 3] :
            y_data = cv2.resize(y_data , (512 ,512))

        # turn to torch (typecasting)
        x_data, y_data = torch.from_numpy(x_data).type(torch.float32), torch.from_numpy(y_data).type(torch.float32)
        return x_data , y_data  #torch

        #torch or numpy to chose

        
        
    
        


In [6]:
 
#same patient (same index)
total_data_list = []
total_target_list =[]  
for name in name_set :
    name1 = name + ".jpg"
    name2 = name + ".png"
    #total_data_list.append(non_T1C_data + name + '.jpg')
    #total_target_list.append(non_T1C_target + name + '.png')
    total_data_list.append(os.path.join(non_T1C_data , name1))
    total_data_list.append(os.path.join(non_T1_data , name1))
    total_data_list.append(os.path.join(non_Flair_data , name1))
    total_data_list.append(os.path.join(non_T2_data , name1))
    total_target_list.append(os.path.join(non_T1C_target , name , name2))
    total_target_list.append(os.path.join(non_T1_target , name , name2))
    total_target_list.append(os.path.join(non_Flair_target , name , name2))
    total_target_list.append(os.path.join(non_T2_target , name , name2))

    total_data_list.append(os.path.join(PR_T1C_data , name1))
    total_data_list.append(os.path.join(PR_T1_data , name1))
    total_data_list.append(os.path.join(PR_Flair_data , name1))
    total_data_list.append(os.path.join(PR_T2_data , name1))
    total_target_list.append(os.path.join(PR_T1C_target , name , name2))
    total_target_list.append(os.path.join(PR_T1_target , name , name2))
    total_target_list.append(os.path.join(PR_Flair_target , name , name2))
    total_target_list.append(os.path.join(PR_T2_target , name , name2))
print(total_data_list[10])
img10 = cv2.imread(total_data_list[10])
img10_2 = cv2.imread("./data/PSPF_voc_data/non_PR/Flair/JPEGImages/28358688.jpg")
cv2.imshow("10" , img10)
cv2.imshow("10_2" , img10_2)
cv2.waitKey(0)
cv2.destroyAllWindows()
print("**********")
print(total_target_list)    

train_data_list = total_data_list[0 : int(len(total_data_list)*0.8)] #80%data
train_target_list =  total_target_list[0 : int(len(total_target_list)*0.8) ] #80%target

test_data_list = total_data_list[int(len(total_data_list)*0.8) : len(total_data_list)] #20%data
test_target_list = total_target_list[int(len(total_target_list)*0.8) : len(total_target_list)] #20%target

"""#img visualize
count , count2 = 0 , 0
for img1 in train_data_list:
    img1 = cv2.imread(img1)
    plt.imshow(img1)
    plt.show()
    print(img1.shape)
    count += 1
    if count > 10 : break
for img2 in target_data_list:
    img2 = cv2.imread(img2)
    plt.imshow(img2)
    plt.show()
    print(img2.shape)
    count2 += 1
    if count2 > 10: break
"""

#std and mean calculated
#total dataset
total_dataset = Total_dataset(total_data_list ,
                              total_target_list,
                              transform= None
)
total_dataloader = DataLoader(total_dataset , batch_size= len(total_dataset) , shuffle= False)
x , y = next(iter(total_dataloader))
print("x y mean0" ,x[0].mean() , y[0].mean()) #ch1
print("x y mean1" ,x[1].mean() , y[1].mean()) #ch2
print("x y mean2" ,x[2].mean() , y[2].mean()) #ch3




#目前尚未 rescale 跟 normalize
train_dataset = Total_dataset(train_data_list,
                             train_target_list ,
                             transform= T.Compose([T.ToTensor()])
                            ) 
#transform should add                
training_dataloader = DataLoader(train_dataset,
                                 batch_size=10,
                                 shuffle=True)

test_dataset = Total_dataset(test_data_list ,
                            test_target_list ,
                            transform= None)
                
testing_dataloader = DataLoader(test_dataset,
                                 batch_size=10,
                                 shuffle=True)
        
train_x, train_y = next(iter(training_dataloader))
test_x , test_y = next(iter(testing_dataloader))

#img rescale
"""imgs_train = train[0:int (n_input*0.8), :, :,:] / 255 
imgs_test = train[int (n_input*0.8):n_input, :, :,:] / 255  
imgs_mask_train = label[0:int (n_input*0.8), :, :,:]  / 255 
imgs_test_label = label[int (n_input*0.8):n_input, :, :,:]  / 255 """


print(f'train_x = shape: {train_x.shape}; type: {train_x.dtype}')
print(f'train_y = shape: {train_y.shape}; type: {train_y.dtype}')
print(f'train_x = min: {train_x.min()}; max: {train_x.max()}')
print(f'train_y = shape: {train_y.shape}; class: {train_y.unique()}; type: {train_y.dtype}')




./data/PSPF_voc_data/non_PR/Flair/JPEGImages\29285707.jpg


AttributeError: module 'cv2.cv2' has no attribute 'waitkey'

In [ ]:
import os 
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as keras
#from data import *
from PIL import Image
import scipy
import cv2
from matplotlib import pyplot as plt
from scipy.stats.mstats import zscore

In [ ]:
#hyperparameters

In [1]:
class Unet_model():
    def __init__(self , img_row = 512 , img_col = 512):
        self.img_row = img_row
        self.img_col = img_col
    
    def get_model(self):

        inputs = Input((self.img_rows, self.img_cols,3)) #3 channels
		
        conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
		#print "conv1 shape:",conv1.shape
        conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
		#print "conv1 shape:",conv1.shape
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
		#print "pool1 shape:",pool1.shape

        conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
		#print "conv2 shape:",conv2.shape
        conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
		#print "conv2 shape:",conv2.shape
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
		#print "pool2 shape:",pool2.shape

        conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
		#print "conv3 shape:",conv3.shape
        conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
		#print "conv3 shape:",conv3.shape
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
		#print "pool3 shape:",pool3.shape

        conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
        conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
        drop4 = Dropout(0.5)(conv4)
        pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

        conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
        conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
        drop5 = Dropout(0.5)(conv5)

        up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
        merge6 = merge([drop4,up6], mode = 'concat', concat_axis = 3)
        conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
        conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

        up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
        merge7 = merge([conv3,up7], mode = 'concat', concat_axis = 3)
        conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
        conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

        up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
        merge8 = merge([conv2,up8], mode = 'concat', concat_axis = 3)
        conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
        conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

        up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
        merge9 = merge([conv1,up9], mode = 'concat', concat_axis = 3)
        conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
        conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
        conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
        conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

        #group layers into model
        model = Model(input = inputs, output = conv10)

        model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

        return model

    def train(self):
        
            print("loading data")
            
            train = np.load('../../../../media/training3.npy').reshape([4507, 256, 256, 4]) 
            label = np.load('../../../../media/label3.npy').reshape([4507, 256, 256, 4])
            
            
            n_input = train.shape[0]
            n_channel = train.shape[3]
            
            imgs_train = train[0:int (n_input*0.8), :, :,:] / 255
            imgs_test = train[int (n_input*0.8):n_input, :, :,:] / 255
            imgs_mask_train = label[0:int (n_input*0.8), :, :,:]  / 255
            imgs_test_label = label[int (n_input*0.8):n_input, :, :,:]  / 255
            
            #print("merging channels")
            #imgs_train = formSingleChannel(imgs_train)
            #imgs_test1 = formSingleChannel(imgs_test)
            
            print("loading data done")
            model = self.get_unet()
            #model = load_model("unet.hdf5")
            print("got unet")
            
            model_checkpoint = ModelCheckpoint('../../../../media/unet.hdf5', monitor='loss',verbose=1, save_best_only=True)
            print('Fitting model...')
            model.fit(imgs_train, imgs_mask_train, batch_size=20, nb_epoch=10, verbose=1,validation_split=0.25, shuffle=True, callbacks=[model_checkpoint])
            
            print('predict test data')
            imgs_mask_test = model.predict(imgs_test, batch_size=1, verbose=1)
            np.save('../../../../media/imgs_mask_test.npy', imgs_mask_test)

        

IndentationError: expected an indented block (Temp/ipykernel_10660/619793162.py, line 3)